In [1]:
import glob
import json
import os

import anthropic

In [2]:
conversations = []

paths = glob.glob('./res/297.SNS 데이터 고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0

for path in paths:
    with open(path, 'r', encoding='UTF-8') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break
                
target_count = 30
count = 20

for path in paths:
    with open(path, 'r', encoding='UTF-8') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break
                
count

30

In [3]:
prompt = f"""You are given a conversation among three users below. Imagine what the users discussed prior to the conversation.
Please write the previous content, strictly following the tone and format of given conversation, with at least 3000 characters.

{conversations[-2]}
"""

In [4]:
print(conversations[-2])

P01: 요즘 제트플립 새로 나와서 난리더라!
P01: 우리 사람들 역시 뭐 하나 있으면 난리 치는 거 절레절레야...
P02: 키키 아니... 그럴 수도 있지...
P02: 찔려서 그러는 건 아니야  ^^
P03: 키키 곧 아이폰 13 나온대
P03: 나나야 준비됐지?
P01: 키키 그냥 1년 더 기다려서 14 살까 고민 중~
P02: 나나야 너 폰은 1년 더 안 되겠던데?
P02: 카메라... 그때 한 장 찍으니까 멈추던 걸?
P03: 이번 년도까지 쓰면 몇 년 쓴 거야?
P01: 키키 이제 햇수로는 3년 18년도에 샀어!
P01: 그니까 카메라 때문에 고민이야
P01: 안 망가졌으면 무조건 안 바꿨다!
P02: 나랑 진짜 똑같네 키키
P02: 난 13은 다 소문이긴 하겠지만 별로 안 가고 싶어  키키
P02: 그냥 12 싸게 지원해준다고 해서 그거 살까 고민 중
P03: 나 12쓰는데 좋아
P03: 가볍고 이쁨!
P03: 그리고 이제 13도 나온다니까 처음 가격보다도 싸겠넹
P03: 바꿔 바꿔
P01: 근데 언니 12 샀다가 뭐 맘에 안 들어서 팔았다 한 거 아냐?
P02: 웅웅 그 충전 단자가 씨 타입이 아니라서...
P02: 13 일단 나오는 거 보긴 할 건데 안 끌려!
P02: 그리고 14 나올 때까지 기다리면 내 폰이 못 버틸 것 같아
P03: 맞아 둘 것도 바꾸긴 해야 돼
P03: 나도 6단계 업그레이드 키키
P01: 14로 바꾸고 싶은데 13 쓰다가 바꿀지 지금 거 계속 쓰다가 바꿀지 고민 좀 해봐야지...
P02: 키키 14도 어떻게 나올지 상상이 안 가
P02: 제발 인덕션만 좀 어떻게 없애주라...
P03: 키키 맞아 글고 13 아직 나오지도 않았잖아
P03: 생각보다 이쁘게 나올 수도!
P01: 맞아 일단 13 나오는 거 보고 생각할래~
P02: 아 그리고 이제 선 이어폰 없어져서 너무 싫어 ㅠㅠ
P03: 근데 또 없으면 없는 대로 쓰게 되더라
P03: 옛날에 나도 없어서 아쉬웠는데 막상 쓰면 생각도 안 나 키키
P01: 그치 그치 근데

In [5]:
import keyring
ANTHROPIC_API_KEY = keyring.get_password('anthropic', 'key_for_windows')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY
)

message = client.messages.create(
    model='claude-3-5-sonnet-20240620',
    max_tokens=4096,
    temperature=0.0,
    messages=[
        {'role': 'user', 'content':prompt}
    ]
)

message.usage       # API 호출 비용 대략 50원

Usage(input_tokens=1345, output_tokens=1978)

In [6]:
message.content[0].text.split('\n\n')

["Here's an imagined previous conversation leading up to the given dialogue, maintaining the tone and format:",
 "P02: 안녕하세요 여러분! 오늘 날씨 정말 좋네요~\nP01: 네 맞아요! 오랜만에 맑은 하늘이에요.\nP03: 맞아요~ 산책하기 딱 좋은 날씨예요!\nP02: 그러게요. 요즘 집에만 있어서 답답했는데 좋네요.\nP01: 저도 그래요. 코로나 때문에 밖에 나가기가 꺼려졌었거든요.\nP03: 맞아요. 다들 백신 맞으셨나요?\nP02: 네, 저는 2차까지 다 맞았어요.\nP01: 저도 2차 맞았어요! 부작용은 없으셨나요?\nP03: 저는 1차만 맞았는데, 팔이 좀 아팠어요.\nP02: 저는 괜찮았어요. 그냥 좀 피곤했던 정도?\nP01: 저도 비슷했어요. 다행히 큰 부작용은 없었네요.\nP03: 다행이에요. 저도 2차 맞을 때 걱정되긴 해요.\nP02: 괜찮을 거예요. 사람마다 다르니까요.\nP01: 그러게요. 그나저나 요즘 뭐하고 지내세요?\nP03: 저는 요즘 넷플릭스 보면서 시간 보내고 있어요.\nP02: 오~ 뭐 재밌는 거 있나요?\nP03: '오징어 게임' 봤어요. 정말 재밌더라고요!\nP01: 아 저도 봤어요! 진짜 몰입해서 봤죠.\nP02: 저도 봤어요! 마지막에 충격적이었죠...\nP03: 맞아요. 결말이 좀 슬펐어요.\nP01: 그래도 시즌 2가 나온다고 하니 기대되네요.\nP02: 정말요? 그거 좋은 소식이네요!\nP03: 맞아요. 기대돼요. 그리고 요즘 '킹덤' 시리즈도 보고 있어요.\nP01: 아 좀비물이죠? 저는 무서워서 못 봐요 ㅠㅠ\nP02: 저는 재밌게 봤어요. 한국 사극에 좀비라니 신선하더라고요.\nP03: 맞아요. 독특한 조합이에요.\nP01: 그렇군요. 나중에 용기 내서 한번 봐야겠어요.\nP02: 그래요, 한번 도전해보세요!\nP03: 그나저나 여러분, 요즘 운동은 하고 계세요?\nP01: 저는 집에서 요가 영상 보면서 따라 

In [7]:
lst = message.content[0].text.split('\n\n')[1:]
for l in lst[:25]:
    print(l)

P02: 안녕하세요 여러분! 오늘 날씨 정말 좋네요~
P01: 네 맞아요! 오랜만에 맑은 하늘이에요.
P03: 맞아요~ 산책하기 딱 좋은 날씨예요!
P02: 그러게요. 요즘 집에만 있어서 답답했는데 좋네요.
P01: 저도 그래요. 코로나 때문에 밖에 나가기가 꺼려졌었거든요.
P03: 맞아요. 다들 백신 맞으셨나요?
P02: 네, 저는 2차까지 다 맞았어요.
P01: 저도 2차 맞았어요! 부작용은 없으셨나요?
P03: 저는 1차만 맞았는데, 팔이 좀 아팠어요.
P02: 저는 괜찮았어요. 그냥 좀 피곤했던 정도?
P01: 저도 비슷했어요. 다행히 큰 부작용은 없었네요.
P03: 다행이에요. 저도 2차 맞을 때 걱정되긴 해요.
P02: 괜찮을 거예요. 사람마다 다르니까요.
P01: 그러게요. 그나저나 요즘 뭐하고 지내세요?
P03: 저는 요즘 넷플릭스 보면서 시간 보내고 있어요.
P02: 오~ 뭐 재밌는 거 있나요?
P03: '오징어 게임' 봤어요. 정말 재밌더라고요!
P01: 아 저도 봤어요! 진짜 몰입해서 봤죠.
P02: 저도 봤어요! 마지막에 충격적이었죠...
P03: 맞아요. 결말이 좀 슬펐어요.
P01: 그래도 시즌 2가 나온다고 하니 기대되네요.
P02: 정말요? 그거 좋은 소식이네요!
P03: 맞아요. 기대돼요. 그리고 요즘 '킹덤' 시리즈도 보고 있어요.
P01: 아 좀비물이죠? 저는 무서워서 못 봐요 ㅠㅠ
P02: 저는 재밌게 봤어요. 한국 사극에 좀비라니 신선하더라고요.
P03: 맞아요. 독특한 조합이에요.
P01: 그렇군요. 나중에 용기 내서 한번 봐야겠어요.
P02: 그래요, 한번 도전해보세요!
P03: 그나저나 여러분, 요즘 운동은 하고 계세요?
P01: 저는 집에서 요가 영상 보면서 따라 하고 있어요.
P02: 저는 가끔 산책 정도만 해요. 운동할 시간이 없네요 ㅠㅠ
P03: 저도 집에서 홈트레이닝 하고 있어요. 요즘 살이 좀 쪘거든요.
P01: 맞아요. 집에만 있다 보니 살이 쉽게 찌더라고요.
P02: 그러게요. 저도 옷이 좀 끼는 것 같

In [8]:
print(len(message.content[0].text.split('\n\n')[1]))

1851


In [9]:
lst = message.content[0].text.split('\n\n')[1:]
lst

["P02: 안녕하세요 여러분! 오늘 날씨 정말 좋네요~\nP01: 네 맞아요! 오랜만에 맑은 하늘이에요.\nP03: 맞아요~ 산책하기 딱 좋은 날씨예요!\nP02: 그러게요. 요즘 집에만 있어서 답답했는데 좋네요.\nP01: 저도 그래요. 코로나 때문에 밖에 나가기가 꺼려졌었거든요.\nP03: 맞아요. 다들 백신 맞으셨나요?\nP02: 네, 저는 2차까지 다 맞았어요.\nP01: 저도 2차 맞았어요! 부작용은 없으셨나요?\nP03: 저는 1차만 맞았는데, 팔이 좀 아팠어요.\nP02: 저는 괜찮았어요. 그냥 좀 피곤했던 정도?\nP01: 저도 비슷했어요. 다행히 큰 부작용은 없었네요.\nP03: 다행이에요. 저도 2차 맞을 때 걱정되긴 해요.\nP02: 괜찮을 거예요. 사람마다 다르니까요.\nP01: 그러게요. 그나저나 요즘 뭐하고 지내세요?\nP03: 저는 요즘 넷플릭스 보면서 시간 보내고 있어요.\nP02: 오~ 뭐 재밌는 거 있나요?\nP03: '오징어 게임' 봤어요. 정말 재밌더라고요!\nP01: 아 저도 봤어요! 진짜 몰입해서 봤죠.\nP02: 저도 봤어요! 마지막에 충격적이었죠...\nP03: 맞아요. 결말이 좀 슬펐어요.\nP01: 그래도 시즌 2가 나온다고 하니 기대되네요.\nP02: 정말요? 그거 좋은 소식이네요!\nP03: 맞아요. 기대돼요. 그리고 요즘 '킹덤' 시리즈도 보고 있어요.\nP01: 아 좀비물이죠? 저는 무서워서 못 봐요 ㅠㅠ\nP02: 저는 재밌게 봤어요. 한국 사극에 좀비라니 신선하더라고요.\nP03: 맞아요. 독특한 조합이에요.\nP01: 그렇군요. 나중에 용기 내서 한번 봐야겠어요.\nP02: 그래요, 한번 도전해보세요!\nP03: 그나저나 여러분, 요즘 운동은 하고 계세요?\nP01: 저는 집에서 요가 영상 보면서 따라 하고 있어요.\nP02: 저는 가끔 산책 정도만 해요. 운동할 시간이 없네요 ㅠㅠ\nP03: 저도 집에서 홈트레이닝 하고 있어요. 요즘 살이 좀 쪘거든요.\nP01: 맞아요. 집에만 있다 보니 살이 쉽

In [10]:
import pickle

with open('./res/conv_long.pickle', 'wb') as f:
    pickle.dump(lst, f)